<a href="https://colab.research.google.com/github/xavirubi/tappx-challenge/blob/master/alpaca_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Talk to Alpaca-LoRA

This notebook contains minimal code for running [Alpaca-LoRA](https://github.com/tloen/alpaca-lora/) for demonstration purposes. Please check the repo for more details.

In [1]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 33.8 MB/s eta 0:00:00
  Installing b

In [2]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-jma6qg4zo12p --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=t

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [3]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [4]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        # print("Response:", output.split("### Response:")[1].strip())
        return ("Response:", output.split("### Response:")[1].strip())

In [ ]:
!pip install -U deep-translator

In [7]:
import json
from deep_translator import GoogleTranslator


f_articles = open('articles.json')
f_videos = open('videos.json')

articles_data = json.load(f_articles)
videos_data = json.load(f_videos)

article_ids = list(article_id for article_id in articles_data.keys())
video_ids = list(video_id for video_id in videos_data.keys())

def article_keywords(id):
	return articles_data[id]["keywords"]

def video_keywords(id):
	return videos_data[id]["keywords"]


# # translated = GoogleTranslator(source='es', target='en').translate(text)

# # article = ' '.join(article_keywords(article_ids[0]))
# # en_article = GoogleTranslator(source='es', target='en').translate(article)
# for i in range(0, len(article_ids)):
#   article = ' '.join(article_keywords(article_ids[i]))
#   en_article = GoogleTranslator(source='es', target='en').translate(article)
#   for j in range(0, len(video_ids)):
#     video = ' '.join(video_keywords(video_ids[j]))
#     en_video = GoogleTranslator(source='es', target='en').translate(video)
#     response = evaluate(f'''give a score form 1 to 100 that reflects how much is this set of words "{en_article}" related to this other set of words "{en_video}"''')
#     # if 'yes' in response[1].lower():
#     print(f'''Article {i}: {article}\nVideo {j}: {video}\n{response[1]}\n\n''')


In [10]:
from deep_translator import GoogleTranslator


def article_text(id):
	return articles_data[id]["text"]


test = GoogleTranslator(source='es', target='en').translate(article_text(article_ids[0]))

print(test)

The sad story of the Spanish submarine that did not float | Europe | D.W.

The S-80 is the latest jewel of the Spanish Navy. The problem is that it didn't float. That was solved by enlarging it. The problem now is that it doesn't fit in your dock. And it is very expensive, in addition. Spain's submarine fleet had to be renewed and for this the Ministry of Defense decided to invest 2,132 million euros to build four S-80 submersibles. Everything was going well until the machine was put to the test, in 2013. Then an error in the design was discovered that could put the buoyancy of the submarine at risk. To put it in Christian: there was a possibility that the S-80 Isaac Peral would sink and never return to the surface. The experts did the math and discovered that it had up to 100 tons left over due to "deviations related to the balance of weights. They analyzed the situation and determined that to correct the problem, the submersible had to be enlarged. To be sure, they hired some America

In [ ]:
import json
from deep_translator import GoogleTranslator


f_articles = open('articles.json')
f_videos = open('videos.json')

articles_data = json.load(f_articles)
videos_data = json.load(f_videos)

article_ids = list(article_id for article_id in articles_data.keys())
video_ids = list(video_id for video_id in videos_data.keys())

def article_keywords(id):
	return articles_data[id]["keywords"]

def video_keywords(id):
	return videos_data[id]["keywords"]

def article_text(id):
	return articles_data[id]["text"]

def video_text(id):
	return videos_data[id]["text"]

# translated = GoogleTranslator(source='es', target='en').translate(text)

article = ' '.join(article_text(article_ids[0]))
en_article = GoogleTranslator(source='es', target='en').translate(article)

for i in range(0, len(video_ids)):
  en_video = GoogleTranslator(source='es', target='en').translate(' '.join(video_text(video_ids[i])))
  print(f"{i}.     ")
  evaluate(f'''is this text "{en_article}" related to this other text "{en_video}"''')